In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
data = pd.read_csv('Pokemon.csv')
data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [3]:
legendaries = data[data['Legendary']==True]
commons = data[data['Legendary']==False]

## Legendaries

In [9]:
print('number :', len(legendaries))
print('average total stats :', np.mean(legendaries['Total']))
for stat in ['HP', 'Attack','Defense', 'Sp. Atk', 'Sp. Def', 'Speed']:
    print('average', stat, ':', np.mean(legendaries[stat]))
print('Number of legendaries per type :', Counter(legendaries['Type 1']))
print('Number of legendaries per generation :', Counter(legendaries['Generation']))
#print('IDs :', list(legendaries['#']))

number : 75
average total stats : 630.2666666666667
average HP : 93.08
average Attack : 113.37333333333333
average Defense : 98.93333333333334
average Sp. Atk : 119.73333333333333
average Sp. Def : 105.01333333333334
average Speed : 100.13333333333334
Number of legendaries per type : Counter({'Psychic': 17, 'Dragon': 12, 'Water': 8, 'Fire': 5, 'Electric': 4, 'Rock': 4, 'Steel': 4, 'Ground': 4, 'Normal': 4, 'Grass': 3, 'Ice': 2, 'Ghost': 2, 'Dark': 2, 'Flying': 2, 'Bug': 1, 'Fairy': 1})
Number of legendaries per generation : Counter({5: 20, 3: 18, 4: 16, 6: 8, 1: 7, 2: 6})


## Non-legendaries

In [8]:
print('number :', len(commons))
print('average stats :', np.mean(commons['Total']))
for stat in ['HP', 'Attack','Defense', 'Sp. Atk', 'Sp. Def', 'Speed']:
    print('average', stat, ':', np.mean(commons[stat]))
print('Number of non-legendaries per type :', Counter(commons['Type 1']))
print('Number of non-legendaries per generation :', Counter(commons['Generation']))
#print('IDs :', list(commons['#']))

number : 725
average stats : 414.9131034482759
average HP : 66.79448275862069
average Attack : 75.4455172413793
average Defense : 71.24689655172413
average Sp. Atk : 67.96689655172413
average Sp. Def : 68.47724137931034
average Speed : 64.98206896551724
Number of non-legendaries per type : Counter({'Water': 104, 'Normal': 94, 'Bug': 68, 'Grass': 67, 'Fire': 47, 'Electric': 40, 'Psychic': 40, 'Rock': 40, 'Ghost': 30, 'Dark': 29, 'Poison': 28, 'Ground': 28, 'Fighting': 27, 'Steel': 23, 'Ice': 22, 'Dragon': 20, 'Fairy': 16, 'Flying': 2})
Number of non-legendaries per generation : Counter({1: 159, 5: 145, 3: 142, 4: 105, 2: 100, 6: 74})
